In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy.sparse import lil_matrix
from scipy.sparse.linalg import spsolve
from IPython.display import HTML

# --- 1. Parameters & scaling ---
# We define the physics to be consistent with L=1 and g=9.81
L = 1.0
Nx, Ny = 50, 50  # 50x50 is faster and stable for testing
dx, dy = L/Nx, L/Ny
dt = 0.002       # Small time step for stability

# Target Dimensionless Numbers
Re_target = 1000.0
Pe_target = 200.0

# Derive Physical Viscosity/Diffusivity from target Re/Pe
# Assuming a characteristic velocity U ~ 0.1 m/s (buoyancy driven)
U_char = 0.1 
nu = (U_char * L) / Re_target
D  = (U_char * L) / Pe_target

print(f"Simulation Setup:")
print(f"  Re: {Re_target} -> Kinematic Viscosity (nu): {nu:.6f}")
print(f"  Pe: {Pe_target} -> Diffusivity (D): {D:.6f}")
print(f"  Grid: {Nx}x{Ny}")

# --- 2. Helper Functions ---

def get_k(i, j):
    """Maps 2D indices (i,j) to 1D index k."""
    return i + j * Nx

def solve_rho_upwind(u, v, rho_n, D_coeff, dt):
    N = Nx * Ny
    A = lil_matrix((N, N))
    b = np.zeros(N)

    # Pre-calculate flux/diffusion factors
    # Scaling: we solve for (rho_new - rho_old)/dt + ...
    Fx = dt / dx
    Fy = dt / dy
    Dx = (D_coeff * dt) / (dx**2)
    Dy = (D_coeff * dt) / (dy**2)

    # Centers of cells for velocity
    u_center = 0.5 * (u[:, 0:-1] + u[:, 1:])
    v_center = 0.5 * (v[0:-1, :] + v[1:, :])

    for j in range(Ny):
        for i in range(Nx):
            idx = get_k(i, j)
            
            # --- Boundary Conditions ---
            # Bottom Wall: Fixed rho = 0
            if j == 0:
                A[idx, idx] = 1.0
                b[idx] = 0.0
                continue
            # Top Wall: Fixed rho = 1
            if j == Ny - 1:
                A[idx, idx] = 1.0
                b[idx] = 1.0
                continue

            # --- Interior ---
            u_val = u_center[j, i]
            v_val = v_center[j, i]

            # Upwind Coefficients (Flux + Diffusion)
            # Checks direction of flow to decide which neighbor to use
            ae = -Dx + min(0, u_val * Fx)  # Flow from East?
            aw = -Dx - max(0, u_val * Fx)  # Flow from West?
            an = -Dy + min(0, v_val * Fy)  # Flow from North?
            as_ = -Dy - max(0, v_val * Fy) # Flow from South?

            # Diagonal (Time term + Outgoing fluxes)
            ap = 1.0 - (ae + aw + an + as_)

            # Neighbors
            idx_e = get_k(i+1, j)
            idx_w = get_k(i-1, j)
            idx_n = get_k(i, j+1)
            idx_s = get_k(i, j-1)

            # Assembly
            A[idx, idx] = ap
            
            # X-Direction (Neumann/Insulated Walls)
            if i + 1 < Nx: A[idx, idx_e] = ae
            else: A[idx, idx] += ae # Add back to diagonal (No Flux)
            
            if i - 1 >= 0: A[idx, idx_w] = aw
            else: A[idx, idx] += aw

            # Y-Direction (Neighbors exist because we handled j=0/Ny-1 above)
            if j + 1 < Ny: A[idx, idx_n] = an
            if j - 1 >= 0: A[idx, idx_s] = as_

            b[idx] = rho_n[j, i]

    return spsolve(A.tocsr(), b).reshape((Ny, Nx))

def solve_u_velocities(u, v, p, nu_coeff, dt):
    Nu = Ny * (Nx - 1)
    AU = lil_matrix((Nu, Nu))
    bu = np.zeros(Nu) 

    # Viscous diffusion scaling
    Dx_u = (nu_coeff * dt) / (dx**2)
    Dy_u = (nu_coeff * dt) / (dy**2)

    def get_row_u(i, j):
        return (i - 1) + j * (Nx - 1)

    for j in range(Ny):
        for i in range(1, Nx): 
            row = get_row_u(i, j) 
            
            # Pressure Gradient
            grad_p = (p[j, i] - p[j, i-1]) / dx
            bu[row] = u[j, i] - (dt * grad_p)

            # Advection (Linearized)
            v_avg = 0.25 * (v[j, i] + v[j, i-1] + v[j+1, i] + v[j+1, i-1])
            u_p = u[j, i]
            Cx = (u_p * dt) / (2 * dx)
            Cy = (v_avg * dt) / (2 * dy)

            # Central Difference Coefficients
            ae = Cx - Dx_u
            aw = -Cx - Dx_u
            an = Cy - Dy_u
            as_ = -Cy - Dy_u
            ap = 1.0 + 2*Dx_u + 2*Dy_u

            AU[row, row] = ap
            
            # Neighbors
            idx_e = get_row_u(i+1, j)
            idx_w = get_row_u(i-1, j)
            idx_n = get_row_u(i, j+1)
            idx_s = get_row_u(i, j-1)

            if i + 1 < Nx: AU[row, idx_e] = ae
            if i - 1 >= 1: AU[row, idx_w] = aw
            
            if j + 1 < Ny: AU[row, idx_n] = an
            else: bu[row] -= an * 0 # Wall u=0
            
            if j - 1 >= 0: AU[row, idx_s] = as_
            else: bu[row] -= as_ * 0 # Wall u=0

    u_solved = spsolve(AU.tocsr(), bu)
    u_new = np.zeros_like(u)
    u_new[:, 1:-1] = u_solved.reshape((Ny, Nx-1))
    return u_new

def solve_v_velocities(u, v, p, rho, nu_coeff, dt):
    Nv = Nx * (Ny - 1)
    AV = lil_matrix((Nv, Nv))
    bv = np.zeros(Nv) 

    Dx_v = (nu_coeff * dt) / (dx**2)
    Dy_v = (nu_coeff * dt) / (dy**2)

    def get_row_v(i, j):
        return i + ((j - 1) * Nx)

    for j in range(1, Ny):
        for i in range(Nx): 
            row = get_row_v(i, j) 

            grad_p = (p[j, i] - p[j-1, i]) / dy
            
            # Buoyancy Term (Dimensional: rho * g)
            rho_avg = 0.5 * (rho[j, i] + rho[j-1, i])
            buoyancy_force = rho_avg * 9.81 
            
            # Subtract buoyancy (Heavy fluid falls -> negative V)
            bv[row] = v[j, i] - (dt * grad_p) - (dt * buoyancy_force)

            # Advection
            u_S_L = u[j, i-1] if i-1 >= 0 else 0
            u_N_L = u[j+1, i-1] if i-1 >= 0 else 0
            u_avg = 0.25 * (u[j, i] + u_S_L + u[j+1, i] + u_N_L)
            v_p = v[j, i]

            Cx = (u_avg * dt) / (2 * dx)
            Cy = (v_p * dt) / (2 * dy)

            ae = Cx - Dx_v
            aw = -Cx - Dx_v
            an = Cy - Dy_v
            as_ = -Cy - Dy_v
            ap = 1.0 + 2*Dx_v + 2*Dy_v

            AV[row, row] = ap

            idx_e = get_row_v(i+1, j)
            idx_w = get_row_v(i-1, j)
            idx_n = get_row_v(i, j+1)
            idx_s = get_row_v(i, j-1)

            if i + 1 < Nx: AV[row, idx_e] = ae
            if i - 1 >= 0: AV[row, idx_w] = aw
            
            if j + 1 < Ny: AV[row, idx_n] = an
            else: bv[row] -= an * 0
            
            if j - 1 >= 1: AV[row, idx_s] = as_
            else: bv[row] -= as_ * 0

    v_solved = spsolve(AV.tocsr(), bv)
    v_new = np.zeros_like(v)
    v_new[1:-1, :] = v_solved.reshape((Ny-1, Nx))
    return v_new

def solve_pressure_correction(u_star, v_star, dt):
    N = Nx * Ny
    A = lil_matrix((N, N))
    b = np.zeros(N)

    Ae = dt / dx**2
    Aw = dt / dx**2
    An = dt / dy**2
    As = dt / dy**2
    Ap_base = -(Ae + Aw + An + As)

    for j in range(Ny):
        for i in range(Nx):
            row = get_k(i, j)
            
            # Divergence
            div_u = (u_star[j, i+1] - u_star[j, i]) / dx
            div_v = (v_star[j+1, i] - v_star[j, i]) / dy
            b[row] = -(div_u + div_v)

            A[row, row] = Ap_base

            # Neumann Boundaries (Wall = No Flux)
            if i + 1 < Nx: A[row, get_k(i+1, j)] = Ae
            else: A[row, row] += Ae
            
            if i - 1 >= 0: A[row, get_k(i-1, j)] = Aw
            else: A[row, row] += Aw
            
            if j + 1 < Ny: A[row, get_k(i, j+1)] = An
            else: A[row, row] += An
            
            if j - 1 >= 0: A[row, get_k(i, j-1)] = As
            else: A[row, row] += As

    # Pinning (Reference Pressure = 0 at bottom left)
    ref = get_k(0, 0)
    A[ref, :] = 0
    A[ref, ref] = 1
    b[ref] = 0

    return spsolve(A.tocsr(), b).reshape((Ny, Nx))

def apply_corrections(u_star, v_star, p_old, p_corr, dt):
    p_new = p_old + p_corr
    u_new = u_star.copy()
    v_new = v_star.copy()

    for j in range(Ny):
        for i in range(1, Nx):
            u_new[j, i] -= dt * (p_corr[j, i] - p_corr[j, i-1]) / dx
            
    for j in range(1, Ny):
        for i in range(Nx):
            v_new[j, i] -= dt * (p_corr[j, i] - p_corr[j-1, i]) / dy
            
    return u_new, v_new, p_new

# --- 3. Main Loop ---

def rayleigh_benard_convection(T_total):
    u = np.zeros((Ny, Nx + 1))
    v = np.zeros((Ny + 1, Nx))
    p = np.zeros((Ny, Nx))
    
    # CRITICAL: Add noise to break symmetry
    rho = np.zeros((Ny, Nx)) + 0.05 * np.random.rand(Ny, Nx)
    
    snapshots = []
    steps = int(T_total / dt)
    print(f"Simulating {steps} steps...")

    for k in range(steps):
        # Solve Density (Upwind is safer)
        rho = solve_rho_upwind(u, v, rho, D, dt)
        
        # Solve Velocity (Dimensional parameters)
        u_star = solve_u_velocities(u, v, p, nu, dt)
        v_star = solve_v_velocities(u, v, p, rho, nu, dt)
        
        # Pressure Correction
        p_corr = solve_pressure_correction(u_star, v_star, dt)
        
        # Update
        u, v, p = apply_corrections(u_star, v_star, p, p_corr, dt)
        
        if k % 10 == 0:
            snapshots.append(rho.copy())
            
        if k % 50 == 0: 
            print(f"Step {k}/{steps}")

    return np.array(snapshots)

# --- 4. Animation ---

def create_animation(snapshots, save_video=False, filename="convection.mp4"):
    x = np.linspace(0, L, Nx)
    y = np.linspace(0, L, Ny)
    X, Y = np.meshgrid(x, y)
    
    n_frames = snapshots.shape[0]
    fig, ax = plt.subplots(figsize=(6, 6))
    
    # Use RdBu_r colormap (Red=Heavy/Top, Blue=Light/Bottom)
    cf = ax.pcolormesh(X, Y, snapshots[0], cmap="RdBu_r", vmin=0, vmax=1, shading="auto")
    cbar = fig.colorbar(cf, ax=ax)
    cbar.set_label("Density")
    ax.set_title("Rayleigh-Benard Convection")
    
    def update(frame):
        ax.clear()
        cf = ax.pcolormesh(X, Y, snapshots[frame], cmap="RdBu_r", vmin=0, vmax=1, shading="auto")
        ax.set_title(f"Time: {frame * dt * 10:.3f} s")
        return []

    ani = animation.FuncAnimation(fig, update, frames=n_frames, interval=50, blit=False)
    
    if save_video:
        print(f"Saving {filename}...")
        ani.save(filename, writer='ffmpeg', fps=20, dpi=100)
        print("Saved.")

    plt.close(fig)
    return HTML(ani.to_jshtml())

# --- Run ---
# Run for 2.0 seconds (enough to see plumes form)
rho_data = rayleigh_benard_convection(T_total=2.0)

# Display Animation
create_animation(rho_data, save_video=False)

Simulation Setup:
  Re: 1000.0 -> Kinematic Viscosity (nu): 0.000100
  Pe: 200.0 -> Diffusivity (D): 0.000500
  Grid: 50x50
Simulating 1000 steps...


IndexError: index 50 is out of bounds for axis 0 with size 50